In [1]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1 MB 1.7 kB/s 
     |████████████████████████████████| 8.8 MB 52.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.2.0+cu92 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.2.0+cu92 which 

In [1]:
import torch 
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor 

In [2]:
train = datasets.MNIST(root ='data',download=True,train=True,transform=ToTensor())
dataset = DataLoader(train, 32)

class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1, 32, (3,3)),
        nn.ReLU(),
        nn.Conv2d(32, 64, (3,3)),
        nn.ReLU(),
        nn.Conv2d(64, 64, (3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6), 10)

    )


  def forward(self, x):
    return self.model(x)

clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

if __name__ == "__main__":
  for epoch in range(10):
    for batch in dataset:
      X,y = batch
      X,y = X.to('cuda'),y.to('cuda')
      yhat = clf(X)
      loss = loss_fn(yhat, y)

      opt.zero_grad()
      loss.backward()
      opt.step()
    
    print(f"Epochs:{epoch} loss is = {loss.item()}")
  
  with open('model_state.pt', 'wb') as f: 
        save(clf.state_dict(), f) 


Epochs:0 loss is = 0.0509725883603096
Epochs:1 loss is = 0.006121531128883362
Epochs:2 loss is = 0.002347096800804138
Epochs:3 loss is = 0.00013329088687896729
Epochs:4 loss is = 1.0892748832702637e-05
Epochs:5 loss is = 3.3915042877197266e-05
Epochs:6 loss is = 0.027713343501091003
Epochs:7 loss is = 1.1086463928222656e-05
Epochs:8 loss is = 1.3113021850585938e-06
Epochs:9 loss is = 1.8477439880371094e-06


In [3]:
if __name__ == "__main__":
  with open('model_state.pt', 'rb') as f: 
        clf.load_state_dict(load(f))  

  img = Image.open('img_1.jpg') 
  img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

  print(torch.argmax(clf(img_tensor)))

tensor(2, device='cuda:0')
